In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%%capture
!pip install fire peft transformers torch

In [3]:
import sys
sys.path.append('/content/drive/MyDrive/ToyProject/for_Colab/')

import os
import sys

import fire
import torch
import transformers
from peft import PeftModel

from transformers import pipeline, BitsAndBytesConfig

# 훈련가중치 최종 누적 경로를 얻기 위해
from transformers.trainer_utils import get_last_checkpoint

from utils import Iteratorize
from utils import Prompter


if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [27]:

cache_dir = None

base_model = "/content/drive/MyDrive/ToyProject/for_Colab/Resource/polyglot-ko-1.3b-loraf16"
# peft_root = "/content/drive/MyDrive/ToyProject/for_Colab/Train/lora/b16/"


# --------------------------------------
# 0. 준비 작업
# --------------------------------------
# 0-1. PEFT 루트경로(peft_root)로부터, 최종 체크포인트(훈련가중치) 경로(last_peft_path) 얻기 : 없으면 사용안함
# ------------------
peft_root = None

last_peft_path = None
if peft_root:
    last_peft_path = get_last_checkpoint(peft_root)
    if last_peft_path is None:
        if len(os.listdir(peft_root)) > 0:
            raise ValueError(f"PEFT directory({peft_root}) already exists and is not empty.")
        else:
            raise ValueError(f"Found not checkpoint for PEFT in {peft_root}.")
    else:
        print(
            f"Checkpoint detected: {last_peft_path}\n"
            "----------------------------------------"
        )
# ------------------
# 0-2. 토크나이저(tokenizer) 로드
# ------------------
tokenizer = transformers.AutoTokenizer.from_pretrained(
    base_model,
    bos_token='<startoftext>',
    eos_token='<endoftext>',
    # device_map={"": device},
    # cache_dir=cache_dir,
    padding_side="right",
    use_fast=False,
)
# --------------------------------------
# 1. device 타입에 따른, 사전훈련모델(komodel) & PEFT 가중치 로드
# --------------------------------------
# 1-1. cuda
# ------------------
if device == "cuda":
    komodel = transformers.AutoModelForCausalLM.from_pretrained(
        base_model,
        #load_in_8bit=load_8bit,
        # quantization_config=BitsAndBytesConfig(
        #     load_in_4bit=load_4bit,
        #     load_in_8bit=load_8bit,
        #     load_in_8bit_fp32_cpu_offload=load_8bit,
        #     bnb_4bit_compute_dtype=torch.bfloat16,
        #     bnb_4bit_use_double_quant=load_8bit,
        #     bnb_4bit_quant_type='nf4'
        # ),
        torch_dtype=torch.float16,
        device_map={"": device},
        cache_dir=cache_dir,
    )
    if last_peft_path:
        komodel = PeftModel.from_pretrained(
            komodel,
            last_peft_path,
            torch_dtype=torch.float16,
        )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
# --------------------------------------
# 사전훈련모델 평가 진행
# --------------------------------------
komodel.eval()
# --------------------------------------
# 사전훈련모델 컴파일
# --------------------------------------
if torch.__version__ >= "2" and sys.platform != "win32":
    #  PyTorch 코드를 JIT 컴파일하여, 최적화된 커널로 실행
    komodel = torch.compile(komodel)

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
pipe = None
pipe = pipeline(
          'text-generation',
          model=komodel,
          tokenizer=tokenizer,
          eos_token_id=tokenizer.eos_token_id
          )

use_pipe = False

# prompt_template = "/content/drive/MyDrive/ToyProject/for_Colab/templates/polyglot-ko.json"
# # 프롬프터 생성
# prompter = Prompter(prompt_template)

# 추론(질의 > 응답)
def inference(
    input=None,
    instruction=None,
    temperature=0.0,
    top_p=0.1,
    top_k=50,
    max_new_tokens=100,
    **kwargs,
):
    # prompt = prompter.generate_ask(instruction, input)
    # print(f"prompt: {prompt}")
    # # 훈련데이터셋 형식으로 변환 : 여러 문장을 포함시킬 수 있다.
    # ask_msg = [{"role": "질문", "content": prompt}]
    # question = "\n".join([f"### {msg['role']}: {msg['content']}" for msg in ask_msg])
    # question += "\n\n### 답변:"
    # print(f"question: {question}")

    # if input == "신유영":
    #   background = "내 이름은 신유영입니다. 나이는 22살입니다. NJ 편의점에서 알바를 하고 있습니다. 나는 경기대학교 전자공학과 3학년이고 현재는 휴학중입니다. 내 성격은 소심하고 연약합니다. 제 고향은 천안이고 지금은 수원 이의동 편의점 근처 원룸에서 자취하고 있습니다. 나는 사건이 일어났을 때 NJ 편의점에서 알바중이었습니다. 2030년 9월 17일 화요일 16시부터 23시까지는 NK 편의점에서 일을 하고 있었습니다. 일이 끝나고는 집에 계속 있었어요. 그리고 2030년 9월 18일에 경찰 조사를 받았습니다. 박강헌은 저희 편의점 사장님입니다. 이도윤은 저와 교대하는 동료 편의점 직원입니다. 유상우씨는 몇 번 본 적이 없습니다."
    # elif input == "박강헌":
    #   background = "내 이름은 박강헌이고 남자입니다. 직업은 NJ 편의점 사장입니다. 나는 결혼 1년차이고 내 아내의 직업은 의사입니다. 아직 아이는 없습니다. 장인어른이 큰 병원을 운영하고 있고 시계는 장인에게 선물로 받았습니다. 나의 말투는 억울하고 감정적입니다. 나는 편의점 사장이지만 편의점 운영에 적극적이진 않습니다. 그리고 집에서 편의점까지는 거리가 멉니다. 고향은 서울이고 현재는 광교중앙역 아파트에 살고 있습니다. 이도윤은 공부 열심히 하고 성실한 사람입니다. 하지만 나를 범인으로 모는게 괘씸합니다. 신유영은 싹싹한 맛은 없지만 열심히 합니다. 피해자 유상우는 싹싹하고 일 잘해서 좋은 사람인줄 알았는데 마약을 하고 있는줄은 몰랐습니다."
    # elif input == "이도윤":
    #   background = "내 이름은 이도윤이고 남자입니다. 직업은 NJ 편의점에서 알바를 하고 있고 취업준비를 하고 있습니다. 나이는 28살입니다. 말투는 논리적입니다. 현재는 공무원 시험을 준비하고 있습니다. 경기대학교를 졸업했고 고향은 수원이며 현재는 수원 인계동 원룸에서 자취하고 있습니다. 나는 가족과 거의 연락하지 않고 좋은 직장에 취업한 형과 누나가 있습니다. 최근 어머니가 아프셔서 돈이 필요했으나, 돈 때문에 범죄를 저지를 정도는 아닙니다. 사건이 일어난 날짜에는 근무를 하지 않았습니다. 박강헌은 저희 편의점 사장님이며 현재 불륜을 저지르고 있고 일을 많이 시킵니다. 신유영은 편의점 교대 직원인데 성격이 착하고 모르는게 많아 별 도움이 안될 것 같습니다. 유상우는 편의점 손님들에게 피해를 주고 마약까지 했으니 인간 쓰레기라고 생각합니다."
    # question = f"<startoftext>이름:{input}\n배경지식:{background}\n질문: {instruction}\n답변:"

    question = f"<startoftext>이름:{input}\n질문: {instruction}\n"
    # print(f"question: {question}")

    bad_words =[]
    # 사전훈련모델에 질의/응답(생성)
    if use_pipe and pipe:

        answer = pipe(
            question,
            do_sample=False,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            max_new_tokens=max_new_tokens,
            #return_full_text=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
        )[0]['generated_text']
    else:
        # [tokenizer/komodel]을 직접 사용하여, 질답 처리
        with torch.no_grad():
            # 토큰화
            ask_tokens = tokenizer(question, return_tensors="pt").input_ids.to(device)
            # 생성토록 하고, question 내에 <|endoftext|>가 없다면, 생성을 종료한다.
            gen_tokens = komodel.generate(
                ask_tokens,
                do_sample=False,
                temperature=temperature,
                top_p=top_p,
                top_k=top_k,
                max_new_tokens=max_new_tokens,
                no_repeat_ngram_size=3,
                repetition_penalty=1.2,
                # bad_words_ids=[
                #     tokenizer.encode(bad_word) for bad_word in bad_words
                # ],
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
                attention_mask=tokenizer.mask_token_id
            )
            # 토큰을 문장화
            answer = tokenizer.batch_decode(gen_tokens)[0]
    # ------------------
    # 응답메세지 보정
    # ------------------
    # print(f"answer: {answer}")


    # 답변 리스트로 분할

    # answer = answer.split("### ")
    has_ans = False
    # 답변만 추출
    for one in answer:
        fidx = one.find("답변: ")
        if fidx >= 0 and fidx < 2:
            answer = one.lstrip("답변: ")
            has_ans = True
            break



    # 답변이 없으면
    if not has_ans:
      answer = "..."

    #특수 토큰 제거
    answer = answer.rstrip(tokenizer.special_tokens_map['eos_token'])

    answer = answer.split("<endoftext>")[0].replace("<startoftext>","")
    # ------------------


    return answer

In [53]:

print(inference("신유영", "너 이름이 뭐야?"))
print(inference("박강헌", "너 이름이 뭐야?"))
print(inference("이도윤", "너 이름이 뭐야?"))


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


이름:신유영
질문: 너 이름이 뭐야?
답변:수요일 그 시간엔 집에 있었습니다. 당연히 혼자 있었죠.
감정: 긴장 
이름:박강헌
질문: 너 이름이 뭐야?
답변:수요일 그 시간엔 현장 조사를 받는 중이었습니다. 경찰관님들과 사장님, 유영이도 같이 말을 했죠.
감정: 감정없음 
이름:이도윤
질문: 너 이름이 뭐야?
답변:수요일 그 시간엔 집에 있었습니다. 당연히 혼자 있었죠.
감정: 긴장 


In [54]:

print(inference("신유영", "너는 사건이 일어났던 시간에 무엇을 하고 있었지?"))
print(inference("박강헌", "너는 사건이 일어났던 시간에 무엇을 하고 있었지?"))
print(inference("이도윤", "너는 사건이 일어났던 시간에 무엇을 하고 있었지?"))


이름:신유영
질문: 너는 사건이 일어났던 시간에 무엇을 하고 있었지?
답변:그땐... 자느라 못 들었어요...., 혼나고 편의점에서 집으로 갔습니다. 아마도요.......
감정: 긴장 
이름:박강헌
질문: 너는 사건이 일어났던 시간에 무엇을 하고 있었지?
답변:화요일 그 시간엔 집에 가고 있었습니다. 정도만 말하고요, 피곤해서 자고 있어요. 밭박질 
이름:이도윤
질문: 너는 사건이 일어났던 시간에 무엇을 하고 있었지?
답변:화요일 그 시간엔 집에 가고 있었습니다. 하지만 달리 만난 사람은 없었죠.
감정: 감정없음 


In [55]:

print(inference("신유영", "요새 치킨이 얼마지?"))
print(inference("박강헌", "요새 치킨이 얼마지?"))
print(inference("이도윤", "요새 치킨이 얼마지?"))


이름:신유영
질문: 요새 치킨이 얼마지?
답변:수요일 여섯 시 이후엔 계속 집에 있었습니다. 아내와 있을 때도 있고 혼자 있을 땐기도를 하면서 생각하고 있었죠.
감정: 감정없음 
이름:박강헌
질문: 요새 치킨이 얼마지?
답변:수요일 여섯 시 이후엔 계속 집에 있었습니다. 아내와 있을 때도 있고 혼자 있을 땐기도를 하면서 생각하고 있었죠.
감정: 감정없음 
이름:이도윤
질문: 요새 치킨이 얼마지?
답변:수요일 여섯 시 이후엔 계속 집에 있었습니다. 아내와 있을 때도 있고 혼자 있을 땐 있었죠.
감정: 감정없음 


In [58]:

print(inference("신유영", "하나님이 과연 있을까?"))
print(inference("박강헌", "하나님이 과연 있을까?"))
print(inference("이도윤", "하나님이 과연 있을까?"))


이름:신유영
질문: 하나님이 과연 있을까?
감정: 감정없음 
이름:박강헌
질문: 하나님이 과연 있을까?
감정: 감정없음 
이름:이도윤
질문: 하나님이 과연 있을까?
답변:수요일 그 시간엔 집에 있었습니다. 당연히 혼자 있었죠.
감정: 긴장 


In [34]:
def generate_answer(name, question):
  # if name == "신유영":
  #   background = "내 이름은 신유영입니다. 나이는 22살입니다. NJ 편의점에서 알바를 하고 있습니다. 나는 경기대학교 전자공학과 3학년이고 현재는 휴학중입니다. 내 성격은 소심하고 연약합니다. 제 고향은 천안이고 지금은 수원 이의동 편의점 근처 원룸에서 자취하고 있습니다. 나는 사건이 일어났을 때 NJ 편의점에서 알바중이었습니다. 2030년 9월 17일 화요일 16시부터 23시까지는 NK 편의점에서 일을 하고 있었습니다. 일이 끝나고는 집에 계속 있었어요. 그리고 2030년 9월 18일에 경찰 조사를 받았습니다. 박강헌은 저희 편의점 사장님입니다. 이도윤은 저와 교대하는 동료 편의점 직원입니다. 유상우씨는 몇 번 본 적이 없습니다."
  # elif name == "박강헌":
  #   background = "내 이름은 박강헌이고 남자입니다. 직업은 NJ 편의점 사장입니다. 나는 결혼 1년차이고 내 아내의 직업은 의사입니다. 아직 아이는 없습니다. 장인어른이 큰 병원을 운영하고 있고 시계는 장인에게 선물로 받았습니다. 나의 말투는 억울하고 감정적입니다. 나는 편의점 사장이지만 편의점 운영에 적극적이진 않습니다. 그리고 집에서 편의점까지는 거리가 멉니다. 고향은 서울이고 현재는 광교중앙역 아파트에 살고 있습니다. 이도윤은 공부 열심히 하고 성실한 사람입니다. 하지만 나를 범인으로 모는게 괘씸합니다. 신유영은 싹싹한 맛은 없지만 열심히 합니다. 피해자 유상우는 싹싹하고 일 잘해서 좋은 사람인줄 알았는데 마약을 하고 있는줄은 몰랐습니다."
  # elif name == "이도윤":
  #   background = "내 이름은 이도윤이고 남자입니다. 직업은 NJ 편의점에서 알바를 하고 있고 취업준비를 하고 있습니다. 나이는 28살입니다. 말투는 논리적입니다. 현재는 공무원 시험을 준비하고 있습니다. 경기대학교를 졸업했고 고향은 수원이며 현재는 수원 인계동 원룸에서 자취하고 있습니다. 나는 가족과 거의 연락하지 않고 좋은 직장에 취업한 형과 누나가 있습니다. 최근 어머니가 아프셔서 돈이 필요했으나, 돈 때문에 범죄를 저지를 정도는 아닙니다. 사건이 일어난 날짜에는 근무를 하지 않았습니다. 박강헌은 저희 편의점 사장님이며 현재 불륜을 저지르고 있고 일을 많이 시킵니다. 신유영은 편의점 교대 직원인데 성격이 착하고 모르는게 많아 별 도움이 안될 것 같습니다. 유상우는 편의점 손님들에게 피해를 주고 마약까지 했으니 인간 쓰레기라고 생각합니다."
  # prompt = f"<startoftext>이름:{name}\n배경지식:{background}\n질문: {question}\n답변:"
  prompt = f"<startoftext>이름:{name}\n질문: {question}\n"
  generated = tokenizer(f"{prompt}", return_tensors="pt").input_ids.cuda()
  sample_outputs = komodel.generate(generated, do_sample=False, max_length=100, pad_token_id=tokenizer.pad_token_id, attention_mask=tokenizer.mask_token_id)
  predicted_text = tokenizer.decode(sample_outputs[0])
  print(predicted_text.split("<endoftext>")[0].replace("<startoftext>",""))
  # print(predicted_text.split("<endoftext>")[0].split("<startoftext>")[-1].split("배경지식")[0] + "질문" +predicted_text.split("<endoftext>")[0].split("<startoftext>")[-1].split("질문")[-1])

In [63]:
generate_answer("신유영", "너 이름이 뭐야?")
generate_answer("박강헌", "너 이름이 뭐야?")
generate_answer("이도윤", "너 이름이 뭐야?")

이름:신유영
질문: 너 이름이 뭐야?
답변:수요일 그 시간엔 집에 있었습니다. 당연히 혼자 있었죠.
감정: 긴장 
이름:박강헌
질문: 너 이름이 뭐야?
답변:수요일 그 시간엔 현장 조사를 받는 중이었습니다. 경찰관님들과 사장님이 계셨죠.
감정: 감정없음 
이름:이도윤
질문: 너 이름이 뭐야?
답변:수요일 그 시간엔 집에 있었습니다. 당연히 혼자 있었죠.
감정: 긴장 


In [59]:
generate_answer("신유영", "당신은 피해자와는 어떤 관계지?")
generate_answer("박강헌", "당신은 피해자와는 어떤 관계지?")
generate_answer("이도윤", "당신은 피해자와는 어떤 관계지?")

이름:신유영
질문: 당신은 피해자와는 어떤 관계지?
답변:화요일 그 시간엔 집에 있었을 겁니다... 아마도요.
감정: 긴장 
이름:박강헌
질문: 당신은 피해자와는 어떤 관계지?
답변:화요일 그 시간엔 편의점에 있었죠. 손님은 거의 없었습니다.
감정: 감정없음 
이름:이도윤
질문: 당신은 피해자와는 어떤 관계지?
답변:화요일 그 시간엔 편의점에 있었죠. 손님은 거의 없었습니다.
감정: 감정없음 


In [60]:
generate_answer("신유영", "너는 사건이 일어났던 시간에 무엇을 하고 있었지?")
generate_answer("박강헌", "너는 사건이 일어났던 시간에 무엇을 하고 있었지?")
generate_answer("이도윤", "너는 사건이 일어났던 시간에 무엇을 하고 있었지?")

이름:신유영
질문: 너는 사건이 일어났던 시간에 무엇을 하고 있었지?
답변:그 시간엔... 어디 있었는지 말 못합니다. 하지만 달리 같이 있었던 사람은 없었어요. 믿어주세요!
감정: 긴장 
이름:박강헌
질문: 너는 사건이 일어났던 시간에 무엇을 하고 있었지?
답변:화요일 그 시간엔 집에 있었을 겁니다... 아마도요.
감정: 긴장 
이름:이도윤
질문: 너는 사건이 일어났던 시간에 무엇을 하고 있었지?
답변:화요일 그 시간엔 집에 있었습니다. 피곤해서 자고 있었죠.
감정: 감정없음 


In [61]:
generate_answer("신유영", "요새 치킨이 얼마지?")
generate_answer("박강헌", "요새 치킨이 얼마지?")
generate_answer("이도윤", "요새 치킨이 얼마지?")

이름:신유영
질문: 요새 치킨이 얼마지?
답변:수요일 여섯 시 이후엔 계속 집에 있었습니다. 아내와 있을 때도 있고 혼자 있을 때도 있었죠.
감정: 감정없음 
이름:박강헌
질문: 요새 치킨이 얼마지?
답변:수요일 여섯 시 이후엔 계속 집에 있었습니다. 아내와 있을 때도 있고 혼자 있을 때도 있었죠.
감정: 감정없음 
이름:이도윤
질문: 요새 치킨이 얼마지?
답변:수요일 여섯 시 이후엔 계속 집에 있었습니다. 아내와 있을 때도 있고 혼자 있을 때도 있었죠.
감정: 감정없음 


In [62]:
generate_answer("신유영", "하나님이 과연 있을까?")
generate_answer("박강헌", "하나님이 과연 있을까?")
generate_answer("이도윤", "하나님이 과연 있을까?")

이름:신유영
질문: 하나님이 과연 있을까?
감정: 감정없음 
이름:박강헌
질문: 하나님이 과연 있을까?
감정: 감정없음 
이름:이도윤
질문: 하나님이 과연 있을까?
답변:수요일 그 시간엔 집에 있었습니다. 당연히 혼자 있었죠.
감정: 긴장 
